In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2024 -  United Kingdom Research and Innovation
#  Copyright 2024 -  The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#  Authored by:    Margaret Duff (STFC-UKRI)
#                  Laura Murgatroyd (STFC-UKRI)
#                  Franck Vidal (STFC-UKRI)
# 

# CIL Preconditioner and step size methods demonstration 

This notebook explores the impact of selecting different step sizes for gradient descent algorithms. We’ll review the available step size methods in the Core Imaging Library (CIL) and guide you through building custom step size strategies tailored to specific needs.

Additionally, the notebook introduces preconditioners—tools that transform a problem into a form more suitable for efficient numerical solving.

CIL’s step size methods and preconditioners offer substantial flexibility, enabling users to adapt and design algorithms customized to their unique optimisation challenges.


In [ ]:
from cil.utilities import dataexample
from cil.utilities.display import show2D
from cil.processors import TransmissionAbsorptionConverter, Slicer
import numpy as np
import matplotlib.pyplot as plt 
from cil.plugins.astra import ProjectionOperator, FBP
from cil.optimisation.algorithms import GD
from cil.optimisation.functions import LeastSquares, L2NormSquared
from cil.optimisation.operators import MatrixOperator
from cil.optimisation.utilities import callbacks, StepSizeMethods, preconditioner, ArmijoStepSizeRule
from cil.framework import  VectorData


# set up default colour map for visualisation
cmap = "gray"

# set the backend for FBP and the ProjectionOperator
device = 'gpu'

## Load Data

In this example, we utilize CIL's simulated sphere data. To accelerate computations in this notebook, we extract a 2D slice from the 3D dataset. Additionally, we select a subset of angles to create a limited-angle reconstruction scenario. We will then compare the ground truth data with a filtered back projection (FBP) reconstruction under these limited-angle conditions.

In [ ]:
ground_truth = dataexample.SIMULATED_SPHERE_VOLUME.get()

data = dataexample.SIMULATED_CONE_BEAM_DATA.get()

data = data.get_slice(vertical='centre')
ground_truth = ground_truth.get_slice(vertical='centre')

absorption = TransmissionAbsorptionConverter()(data)
absorption = Slicer(roi={'angle':(0, -1, 5)})(absorption)

absorption.reorder('astra')
ig = absorption.geometry.get_ImageGeometry()
recon = FBP(ig, absorption.geometry, device=device)(absorption)
show2D([ground_truth, recon], title = ['Ground Truth', 'FDK Reconstruction'], origin = 'upper', num_cols = 2)


## Gradient descent with a fixed step size 

We first consider regularising this limited angle CT reconstruction problem with L2 norm squared (also called Tikhonov) regularisation:
$$ \arg\min_x \|Ax-y\|_2^2 + \alpha \|x\|_2^2 $$ 
where $x$ is the image we wish to reconstruct, $A$ the forward CT operator and $y$ the measured data. The regularisation parameter $\alpha$ is chosen to balance the first, data discrepancy, term and the second, regularisation, term. 

As a starting point, consider solving this optimisation problem with an arbitrary fixed step size, 1e-6. 

In [ ]:
alpha=0.1   
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=absorption.geometry)

F = 0.5*LeastSquares(A = A, b = absorption)+ alpha*L2NormSquared()
algo_GD_fixed=GD(initial=ig.allocate(0), objective_function=F, step_size=1e-6)
algo_GD_fixed.run(50)
show2D([ground_truth, recon, algo_GD_fixed.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)


We now plot the objective value, $\|Ax-y\|_2^2 + \alpha \|x\|_2^2$, against iteration number to look at the speed of convergence of this algorithm. 

In [ ]:
plt.plot(range(2,51),algo_GD_fixed.objective[2:], label='Fixed step size = 1e-6')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Objective value')

Now consider a more sensible choice of fixed step size, the reciprocal of the Lipschitz constant of $\|Ax-y\|_2^2 + \alpha \|x\|_2^2$. The **Lipschitz constant** is a value that quantifies the maximum rate of change of a function. For a function $ f $ to be Lipschitz continuous, there must exist a constant $ L $ (the Lipschitz constant) such that for any two points $ x $ and $ y $, the inequality $ |f(x) - f(y)| \leq L |x - y| $ holds. 

In optimization, the Lipschitz constant is often used to set a safe step size for gradient descent. In CIL, where it is available or easily calculable, the Lipschitz constant of a function is stored as property of the CIL function, for more info see the documentation: https://tomographicimaging.github.io/CIL/nightly/optimisation/#cil.optimisation.functions.Function.L

In [ ]:

algo_GD_lip=GD(initial=ig.allocate(0), objective_function=F, step_size=1/F.L )
algo_GD_lip.run(50)
show2D([ground_truth, recon, algo_GD_lip.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)


We compare the arbitrary step size and the step size chosen using the Lipschitz constant by comparing the object value we are minimising against iteration number. The quicker the decrease, the faster the algorithm is converging. Comparing the two step size choices, we can see that the reciprocal of the Lipschitz constant provides faster convergence rates: 

In [ ]:
plt.plot(range(2,51),algo_GD_fixed.objective[2:], label='Fixed step size = 1e-6')
plt.plot(range(2,51),algo_GD_lip.objective[2:], label='Fixed step size = 1/L')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Objective value')

## Gradient descent default behaviour 

The default behaviour of gradient descent is to use the Armijo step size rule. This is a "backtracking" line search method that iteratively reduces the step size until a sufficient decrease in the objective function is achieved. The Armijo rule ensures that the step size chosen at each iteration satisfies the condition:

$$ f(x_k + \alpha_k \nabla f(x_k)) \leq f(x_k) + c \alpha_k \nabla f(x_k)^T f(x_k) $$

where $ f $ is the objective function, $ x_k $ is the current point, $ \alpha_k $ is the step size, $\nabla f(x_k)$ is the search direction, and $c $ is a constant typically chosen in the interval $ (0, 1) $. This condition guarantees that the step size provides a sufficient decrease in the objective function, balancing between making progress and ensuring stability in the optimization process.


In [ ]:
alpha=0.1
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=absorption.geometry)

F = 0.5*LeastSquares(A = A, b = absorption)+ alpha*L2NormSquared()
algo_default=GD(initial=ig.allocate(0), objective_function=F)
try:
    algo_default.run(50)
except ValueError as e:
    print(e)
show2D([ground_truth, recon, algo_default.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)


This does not work because in 40 iterations, the Armijo step size rule has not found a suitable step size. We define the step size rule explicitly and can alter the number of iterations in the step size rule to allow it to run without error: 

In [ ]:
alpha=0.1
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=absorption.geometry)

F = 0.5*LeastSquares(A = A, b = absorption)+ alpha*L2NormSquared()
step_size_rule=ArmijoStepSizeRule(max_iterations=50)
algo_default=GD(initial=ig.allocate(0), objective_function=F, step_size=step_size_rule)
algo_default.run(50)
show2D([ground_truth, recon, algo_default.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)


We again plot the objective values against iteration numbers to compare the convergence rates: 

In [ ]:
plt.plot(range(2,51),algo_GD_fixed.objective[2:], label='Fixed step size = 1e-6')
plt.plot(range(2,51),algo_GD_lip.objective[2:], label='Fixed step size = 1/L')
plt.plot(range(2,51),algo_default.objective[2:], label='Armijo rule')
plt.legend()

plt.xlabel('Iteration')
plt.ylabel('Objective value')

We see faster convergence for the default step size rule, with the Armijo backtracking. 

## Gradient descent custom step size rule 


In this example we create a custom `shrinking_step_size` rule, inheriting from the `StepSizeMethods.StepSizeRule` base class. This class defines a polynomial step size rule calculated as $$\alpha_t = \frac{\alpha_0}{(1 + k t)^p}$$ where $\alpha_t$ is the step size at iteration 
$t$, $\alpha_0$ is the initial step size, $k$ is the decay constant that controls the rate of decay and $p$ is the power determining the decay rate.

The idea is that with a higher initial step size, the algorithm can make large updates early, moving closer to an optimal region quickly and, 
as the step size decreases, the algorithm takes smaller steps, helping it settle into a more precise region near the optimum without overshooting. 

We build a custom step size rule in CIL to implement this. We define both a constructor and a `get_step_size` method:

Constructor:
- `__init__(self, initial=0.1, constant=0.1)`: Initializes the step size rule with an initial step size and a shrinkage factor.
  - `initial` (float): The initial step size to be used in the first iteration. Default is `0.1`.
  - `constant` (float): A constant that controls the rate of decay, the larger the number the quicker the decay. Default is `0.01`.
  - `power` (float): An integer that controls the type of decay e.g. `power = 1` is linear decay, `power = 2` is quadratic decay and `0 < power < 1` is sublinear decay. The larger the value of the power, the quicker the decay.  Default is `1`

Methods:
- `get_step_size(self, algorithm)`: Computes the step size for the current iteration of the algorithm.
  - `algorithm` (object): The optimization algorithm instance, which is expected to have an `iteration` attribute indicating the current iteration number.
  - Returns: The step size for the current iteration


In [ ]:
class shrinking_step_size(StepSizeMethods.StepSizeRule):
    def __init__(self, initial=0.1, constant=0.01, power=1):
        self.initial=initial
        self.constant=constant
        self.power=power
    
    def get_step_size(self, algorithm):
        return self.initial/((1+self.constant*algorithm.iteration)**self.power)
    
alpha=0.1
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=absorption.geometry)

F = 0.5*LeastSquares(A = A, b = absorption)+ alpha*L2NormSquared()
algo_custom=GD(initial=ig.allocate(0), objective_function=F, step_size=shrinking_step_size(initial=1e-6) )
algo_custom.run(50)
show2D([ground_truth, recon, algo_custom.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)


In [ ]:
plt.plot(range(5,51),algo_GD_fixed.objective[5:], label='Fixed step size = 1e-6')
plt.plot(range(5,51),algo_GD_lip.objective[5:], label='Fixed step size = 1/L')
plt.plot(range(5,51),algo_default.objective[5:], label='Armijio rule')
plt.plot(range(5,51),algo_custom.objective[5:], label='Custom rule')
plt.xlabel('Iteration')
plt.ylabel('Objective value')
plt.legend()

We see that within 15 iterations, the custom step size rule is able to achieve a similar objective value to the Armijo rule, without the additional calculations of the objective and without knowing the Lipschitz constant.

# Preconditioners 

To explain the concept of preconditioners, first look at the following toy problem. 

Consider solving, $$Ax^*=b$$ such that  $b=(0,0)^T$, $A=\begin{pmatrix}
1.0 & 0.0 \\
0.0 & 0.1 
\end{pmatrix}$. 

The unique solution to this is $x^*=(0,0)^T$. 

To visualise this problem we can plot the contours of  $f(x)=\| Ax-b\|_2^2$ for $x=[x_1,x_2]^T$ and see the minimum point, the green star,  at $x^*=[x_1^*,x_2^*]^T=(0,0)$. 

Note: The contour plot is a bit hard to interpret: the more yellow the lines the higher that point is above the minimum, the bottom of the valley. All points on the same line are the same height. 

In [ ]:
def f(x,y):
    return np.linalg.norm(np.matmul(np.array([[1.0,0.0],[0.0, 0.1]]), np.array([x,y]))-np.array([0,0]))


x_ = np.linspace(-0.5, 0.5, num=200)
y_ = np.linspace(-0.5, 0.5, num=200)
x,y = np.meshgrid(x_, y_)

levels = np.zeros((200,200))
for i in range(200):
    for j in range(200):
        levels[i,j]=f(y_[j], x_[i])

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf=ax.plot_surface(x, y, levels, cmap='viridis')

plt.show()

plt.figure()
c = plt.contour(x, y, levels, 30)
plt.scatter([0], [0],marker='*', color='green', s=100)
plt.colorbar()
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Non-preconditioned loss landscape')
plt.show()



The reason this is an ellipse is due to the fact that the matrix $A$ is ill-conditioned, it acts with a greater magnitude in some directions than other directions.

We can find a solution to this inverse problem by gradient descent, minimising $f(x)=\| Ax-b\|_2^2$ and plot the result using a custom callback. 

In [ ]:
class plot_iterates(callbacks.Callback):
    
    def __init__(self, f):
      x_ = np.linspace(-0.5, 0.5, num=200)
      y_ = np.linspace(-0.5, 0.5, num=200)
      x,y = np.meshgrid(x_, y_)

      levels = np.zeros((200,200))
      for i in range(200):
          for j in range(200):
              levels[i,j]=f(y_[j], x_[i])\
                
      plt.contour(x, y, levels, 30)
      plt.colorbar()
      plt.xlabel('x1')
      plt.ylabel('x2')
      plt.scatter([0], [0],marker='*', color='green', s=100)
      self.save_points_x=[]
      self.save_points_y=[]
    
    def __call__(self, algorithm):
      self.save_points_x.append(algorithm.solution.as_array()[0])
      self.save_points_y.append(algorithm.solution.as_array()[1])
      plt.plot(self.save_points_x, self.save_points_y, color='red', marker='o')
  


initial = VectorData(np.array([0.3,0.4]))
b = VectorData(np.array([0.,0.]))
A = MatrixOperator(np.array([[1.,0.0],[0., 0.1]]))
F = 0.5*LeastSquares(A = A, b = b)
cb=plot_iterates(f)
algo=GD(initial=initial, objective_function=F, step_size=1/F.L)
algo.run(50, callbacks=[cb])



We see after an initial large step, the algorithm now slows down as it hits the centre valley of the objective function. 

We precondition by the sensitivity of the matrix $A$ given by a vector $1/(A^T \mathbf{1})$ and can see that this stretches the loss landscape, making it "rounder" with less narrow valleys. 

In [ ]:
precon=preconditioner.Sensitivity(operator=MatrixOperator(np.array([[1,0.0],[0.0, 0.1]])))

In [ ]:
def f_precon(x,y):
    return np.linalg.norm(np.sqrt(precon.array.as_array())*(np.matmul(np.array([[1,0.0],[0.0, 0.1]]), np.array([x,y]))-np.array([0,0])))
plt.figure()

x_ = np.linspace(-0.5, 0.5, num=200)
y_ = np.linspace(-0.5, 0.5, num=200)
x,y = np.meshgrid(x_, y_)

levels = np.zeros((200,200))
for i in range(200):
    for j in range(200):
        levels[i,j]=f_precon(y_[j], x_[i])

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf=ax.plot_surface(x, y, levels, cmap='viridis')

plt.show()

plt.figure()
c = plt.contour(x, y, levels, 30)
plt.scatter([0], [0],marker='*', color='green', s=100)
plt.colorbar()
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('preconditioned loss landscape')
plt.show()


To apply this preconditioning to the gradient descent algorithm, we use the `Sensitivity` preconditioning provided in CIL, initialise this with the operator `A` and then pass this constructor for the Gradient descent algorithm class: `algo_precon=GD(initial=initial, objective_function=F,  preconditioner=preconditioner.Sensitivity(operator=A), step_size=(1/F.L))`.  In the next cell, we run this algorithm and, similarly to above, we can plot the iterates of preconditioned gradient descent:

In [ ]:

initial = VectorData(np.array([0.3,0.4]))
b = VectorData(np.array([0.,0.]))
A = MatrixOperator(np.array([[1.,0],[0., 0.1]]))
F = 0.5*LeastSquares(A = A, b = b)

cb=plot_iterates(f_precon)
algo_precon=GD(initial=initial, objective_function=F,  preconditioner=preconditioner.Sensitivity(operator=A), step_size=(1/F.L))
algo_precon.run(50, callbacks=[cb])





The preconditioned gradient descent converges quicker:

In [ ]:
plt.plot(range(1,51),algo.objective[1:], label='Not preconditioned')
plt.plot(range(1,51),algo_precon.objective[1:], label='Preconditioned')
plt.xlabel('Iteration')
plt.ylabel('Objective value')
plt.legend()

Using a callback, we can see the progress of the algorithm and we see that the initial steps of the preconditioned algorithm get is much closer than in the non-preconditioned case. 

### Preconditioning CT example

We return to our CT example above. Again, we consider least squares with an L2 norm squared regularisation function solved by gradient descent with the Armijo step size rule. We use a callback to keep track of the mean squared error of the reconstructed image compared with the ground truth.

In [ ]:
class mean_squared_error_callback(callbacks.Callback):
    def __init__(self, ground_truth):
        self.ground_truth=ground_truth
        self.errors=[]
    def __call__(self, algorithm):
        self.errors.append(np.linalg.norm(algorithm.solution.as_array()-self.ground_truth.as_array())**2)

In [ ]:
alpha=0.1
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=absorption.geometry)

F = 0.5*LeastSquares(A = A, b = absorption)+ alpha*L2NormSquared()
step_size_rule=ArmijoStepSizeRule(max_iterations=50)
algo=GD(initial=ig.allocate(0), objective_function=F, step_size=step_size_rule) 

cb_no_precon=mean_squared_error_callback(ground_truth)
algo.run(300, callbacks=[cb_no_precon])
show2D([ground_truth, recon, algo.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)

We now add a preconditioner. This time another preconditioner provided in CIL,  `AdaptiveSensitivity`, in each call to the preconditioner the `gradient` is multiplied by $(x+\delta) /(A^T \mathbf{1})$ where $A$ is an operator,  $\mathbf{1}$ is an object in the range of the operator filled with ones. The point $x$ is the current iteration, or a reference image,  and $\delta$ is a small positive float.

In [ ]:
alpha=0.1
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=absorption.geometry)

F = 0.5*LeastSquares(A = A, b = absorption)+ alpha*L2NormSquared()
precon=preconditioner.AdaptiveSensitivity(operator=A)
step_size_rule=ArmijoStepSizeRule(max_iterations=50)
algo_precon=GD(initial=ig.allocate(0), objective_function=F , preconditioner=precon, step_size=step_size_rule)

cb_precon=mean_squared_error_callback(ground_truth)
algo_precon.run(300, callbacks=[cb_precon])
show2D([ground_truth, recon, algo_precon.solution], title = ['Ground Truth', 'FDK Reconstruction', 'L2 regularised solution'], origin = 'upper', num_cols = 3)

We can plot the mean squared error of the reconstructed image compared with the ground truth and how this changes with iteration number:

In [ ]:
plt.plot(range(10,301),cb_no_precon.errors[10:], label='Not preconditioned')
plt.plot(range(10,301),cb_precon.errors[10:], label='Preconditioned')
plt.xlabel('Iteration')
plt.ylabel('Mean squared error')

plt.legend()

We see slightly improved results with the preconditioner.